# MNIST pytorch

### Import package from torch

In [12]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [13]:
if torch.cuda.is_available():
    print(True)
else:
    print(False)

True


### Load test case MNIST

In [14]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

datasets.FashionMNIST()

##### 1、DataLoader 的作用

DataLoader 是 PyTorch 中的一個數據加載器，它可以將一個 Dataset 包裝成可迭代對象，支持批量處理、隨機打亂（shuffle）、多進程數據加載等功能。

##### 2、設定批次大小

批次大小為 64，這意味着每次從數據集中取出 64 個樣本作為一個批次來進行模型的訓練或測試。

In [15]:
batch_size = 64

##### 3、創建數據加載器(DataLoader)

In [16]:
train_dataloader = DataLoader(training_data,batch_size=batch_size)
test_dataloader = DataLoader(test_data,batch_size=batch_size)

##### 4、批量加載數據並查看其形狀

In [17]:
for X,y in test_dataloader:
    print(f"Shape of X [N C H W]:{X.shape}")
    print(f"Shape of y:{y.shape}")
    break

Shape of X [N C H W]:torch.Size([64, 1, 28, 28])
Shape of y:torch.Size([64])


### 建立模型

判別device是否可用

In [18]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using device is {device}")

Using device is cuda


Define model

In [19]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        
        return logits

model = NeuralNetwork().to(device)

print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


### Optimizing the Model Parameters

In [22]:
loss_fun = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [23]:
def train(dataloader, model, loss_fun, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch , (X,y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        pre = model(X)
        loss = loss_fun(pre,y)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grd()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1)*len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()